In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from datetime import datetime

In [2]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1461,tt0414993,35000000,15304890,The Fountain,Hugh Jackman|Rachel Weisz|Ellen Burstyn|Mark M...,Darren Aronofsky,Death is the road to awe,"Spanning over one thousand years, and three pa...",96,Drama|Adventure|Science Fiction|Romance,Regency Enterprises|Warner Bros.|Muse Entertai...,9/6/2006,6.7,2006
1768,tt0452702,19000000,35300645,Vacancy,Kate Beckinsale|Luke Wilson|Ethan Embry|Scott ...,NimrÃ³d Antal,Once you've checked in... The terror begins.,A young married couple becomes stranded at an ...,80,Horror|Thriller,Screen Gems|Hal Lieberman Company,4/20/2007,5.9,2007
1668,tt0349710,60000000,74541707,Ladder 49,Joaquin Phoenix|John Travolta|Jacinda Barrett|...,Jay Russell,Their greatest challenge lies in rescuing one ...,"Under the watchful eye of his mentor, Captain ...",115,Drama|Action|Thriller,Casey Silver Productions|Beacon Communications...,10/1/2004,6.0,2004
873,tt0181689,102000000,358372926,Minority Report,Tom Cruise|Colin Farrell|Samantha Morton|Max v...,Steven Spielberg,The system is perfect until it comes after you.,"John Anderton is a top ""Precrime"" cop in the l...",145,Action|Thriller|Science Fiction|Mystery,DreamWorks SKG|Cruise/Wagner Productions|Ambli...,6/20/2002,6.9,2002
1096,tt0317740,60000000,176070171,The Italian Job,Mark Wahlberg|Charlize Theron|Jason Statham|Ed...,F. Gary Gray,Get in. Get out. Get even.,After being betrayed and left for dead in Ital...,111,Thriller|Action|Crime,Paramount Pictures|De Line Pictures|Working Ti...,5/30/2003,6.6,2003


In [3]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


In [4]:
data.tail()

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1884,tt0120903,75000000,157299717,X-Men,Patrick Stewart|Hugh Jackman|Ian McKellen|Hall...,Bryan Singer,Evolution Begins,"Two mutants, Rogue and Wolverine, come to a pr...",104,Adventure|Action|Science Fiction,Twentieth Century Fox Film Corporation|Donners...,7/13/2000,6.6,2000
1885,tt0192255,22000000,13555988,The Little Vampire,Richard E. Grant|Jonathan Lipnicki|Jim Carter|...,Uli Edel,"They're not just best friends, they're blood b...","Based on the popular books, the story tells of...",95,Horror|Family|Foreign,New Line Cinema,10/27/2000,6.4,2000
1886,tt0131704,76000000,35134820,The Adventures of Rocky & Bullwinkle,Rene Russo|Jason Alexander|Piper Perabo|Randy ...,Des McAnuff,This summer it's not the same old bull.,Rocky and Bullwinkle have been living off the ...,88,Adventure|Animation|Action|Comedy|Family,Universal Pictures|Capella International|KC Me...,6/30/2000,4.0,2000
1887,tt0162983,40000000,36037909,Hanging Up,Meg Ryan|Diane Keaton|Lisa Kudrow|Walter Matth...,Diane Keaton,Every family has a few hang-ups.,A trio of sisters bond over their ambivalence ...,94,Comedy|Drama,Laurence Mark Productions|Columbia Pictures Co...,2/16/2000,5.2,2000
1888,tt0163676,15000000,5217498,The In Crowd,Susan Ward|Lori Heuring|Matthew Settle|Nathan ...,Mary Lambert,What would you do to get in?,A mentally disturbed young woman takes a job a...,105,Thriller,Warner Bros. Pictures,7/19/2000,5.2,2000


# Предобработка

In [7]:
answers = {} # создадим словарь для ответов

In [5]:
# добавим столбец с прибылью
data['profit'] = data['revenue'] - data['budget']

In [6]:
# преобразуем дату выпуска в формат datetime
data.release_date = data.release_date.apply(lambda x:datetime.strptime(x,"%m/%d/%Y"))

In [8]:
# проверяем, что год выпуска совпадает в столбцах release_date и release_year
len(data[data['release_date'].dt.year!=data['release_year']])

0

In [9]:
# преобразуем множественные данные в списки
def splitter (s):
    return s.apply(lambda x:x.split('|'))

In [10]:
data.cast = splitter(data.cast)
data.director = splitter(data.director)
data.genres = splitter(data.genres)
data.production_companies = splitter(data.production_companies)

In [11]:
data.head()

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit
0,tt0369610,150000000,1513528810,Jurassic World,"[Chris Pratt, Bryce Dallas Howard, Irrfan Khan...",[Colin Trevorrow],The park is open.,Twenty-two years after the events of Jurassic ...,124,"[Action, Adventure, Science Fiction, Thriller]","[Universal Studios, Amblin Entertainment, Lege...",2015-06-09,6.5,2015,1363528810
1,tt1392190,150000000,378436354,Mad Max: Fury Road,"[Tom Hardy, Charlize Theron, Hugh Keays-Byrne,...",[George Miller],What a Lovely Day.,An apocalyptic story set in the furthest reach...,120,"[Action, Adventure, Science Fiction, Thriller]","[Village Roadshow Pictures, Kennedy Miller Pro...",2015-05-13,7.1,2015,228436354
2,tt2908446,110000000,295238201,Insurgent,"[Shailene Woodley, Theo James, Kate Winslet, A...",[Robert Schwentke],One Choice Can Destroy You,Beatrice Prior must confront her inner demons ...,119,"[Adventure, Science Fiction, Thriller]","[Summit Entertainment, Mandeville Films, Red W...",2015-03-18,6.3,2015,185238201
3,tt2488496,200000000,2068178225,Star Wars: The Force Awakens,"[Harrison Ford, Mark Hamill, Carrie Fisher, Ad...",[J.J. Abrams],Every generation has a story.,Thirty years after defeating the Galactic Empi...,136,"[Action, Adventure, Science Fiction, Fantasy]","[Lucasfilm, Truenorth Productions, Bad Robot]",2015-12-15,7.5,2015,1868178225
4,tt2820852,190000000,1506249360,Furious 7,"[Vin Diesel, Paul Walker, Jason Statham, Miche...",[James Wan],Vengeance Hits Home,Deckard Shaw seeks revenge against Dominic Tor...,137,"[Action, Crime, Thriller]","[Universal Pictures, Original Film, Media Righ...",2015-04-01,7.3,2015,1316249360


# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [97]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример: 
answers['1'] = '2. Spider-Man 3 (tt0413300)'
# запишите свой вариант ответа
answers['1'] = 'Pirates of the Caribbean: On Stranger Tides (tt1298650)'
# если ответили верно, можете добавить комментарий со значком "+"

In [13]:
# тут пишем ваш код для решения данного вопроса:
data[data.budget==data.budget.max()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit
723,tt1298650,380000000,1021683000,Pirates of the Caribbean: On Stranger Tides,Johnny Depp|PenÃ©lope Cruz|Geoffrey Rush|Ian M...,Rob Marshall,Live Forever Or Die Trying.,Captain Jack Sparrow crosses paths with a woma...,136,"[Adventure, Action, Fantasy]",Walt Disney Pictures|Jerry Bruckheimer Films|M...,2011-05-11,6.3,2011,641683000


ВАРИАНТ 2

In [0]:
# можно добавлять разные варианты решения

# 2. Какой из фильмов самый длительный (в минутах)?

In [99]:
# думаю логику работы с этим словарем вы уже поняли, 
# по этому не буду больше его дублировать
answers['2'] = 'Gods and Generals (tt0279111)'

In [98]:
data[data.runtime==data.runtime.max()]


,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit
1157,tt0279111,56000000,12923936,Gods and Generals,Stephen Lang|Jeff Daniels|Robert Duvall|Kevin ...,Ronald F. Maxwell,The nations heart was touched by...,The film centers mostly around the personal an...,214,Drama|History|War,Turner Pictures|Antietam Filmworks,2003-02-21,5.8,2003,-43076064


# 3. Какой из фильмов самый короткий (в минутах)?





In [10]:
answers['3'] = 'Winnie the Pooh (tt1449283)'

In [100]:
data[data.runtime==data.runtime.min()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit
768,tt1449283,30000000,14460000,Winnie the Pooh,Jim Cummings|Travis Oates|Jim Cummings|Bud Luc...,Stephen Anderson|Don Hall,Oh Pooh.,"During an ordinary day in Hundred Acre Wood, W...",63,Animation|Family,Walt Disney Pictures|Walt Disney Animation Stu...,2011-04-13,6.8,2011,-15540000


# 4. Какова средняя длительность фильмов?


In [104]:
answers['4'] = round(data.runtime.mean())
answers['4']

110

# 5. Каково медианное значение длительности фильмов? 

In [105]:
answers['5'] = round(data.runtime.median())
answers['5']

107

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [106]:
data[data.profit==data.profit.max()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit
239,tt0499549,237000000,2781505847,Avatar,Sam Worthington|Zoe Saldana|Sigourney Weaver|S...,James Cameron,Enter the World of Pandora.,"In the 22nd century, a paraplegic Marine is di...",162,Action|Adventure|Fantasy|Science Fiction,Ingenious Film Partners|Twentieth Century Fox ...,2009-12-10,7.1,2009,2544505847


In [107]:
answers['6'] = 'Avatar (tt0499549)'

# 7. Какой фильм самый убыточный? 

In [108]:
data[data.profit==data.profit.min()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit
1245,tt1210819,255000000,89289910,The Lone Ranger,Johnny Depp|Armie Hammer|William Fichtner|Hele...,Gore Verbinski,Never Take Off the Mask,The Texas Rangers chase down a gang of outlaws...,149,Action|Adventure|Western,Walt Disney Pictures|Jerry Bruckheimer Films|I...,2013-07-03,6.0,2013,-165710090


In [109]:
answers['7'] = 'The Lone Ranger (tt1210819)'

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [111]:
answers['8'] = data[data.profit>0]['imdb_id'].count()
answers['8']

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [126]:
data.query('release_year == 2008').query('revenue==revenue.max()')

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit
599,tt0468569,185000000,1001921825,The Dark Knight,Christian Bale|Michael Caine|Heath Ledger|Aaro...,Christopher Nolan,Why So Serious?,Batman raises the stakes in his war on crime. ...,152,Drama|Action|Crime|Thriller,DC Comics|Legendary Pictures|Warner Bros.|Syncopy,2008-07-16,8.1,2008,816921825


In [127]:
answers['9'] = 'The Dark Knight (tt0468569)'

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [129]:
data.query('2012<=release_year<=2014').query('profit==profit.min()')

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit
1245,tt1210819,255000000,89289910,The Lone Ranger,Johnny Depp|Armie Hammer|William Fichtner|Hele...,Gore Verbinski,Never Take Off the Mask,The Texas Rangers chase down a gang of outlaws...,149,Action|Adventure|Western,Walt Disney Pictures|Jerry Bruckheimer Films|I...,2013-07-03,6.0,2013,-165710090


In [130]:
answers['10'] = 'The Lone Ranger (tt1210819)'

# 11. Какого жанра фильмов больше всего?

In [56]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале
Counter(data.genres.explode()).most_common()

[('Drama', 782),
 ('Comedy', 683),
 ('Thriller', 596),
 ('Action', 582),
 ('Adventure', 415),
 ('Crime', 315),
 ('Romance', 308),
 ('Family', 260),
 ('Science Fiction', 248),
 ('Fantasy', 222),
 ('Horror', 176),
 ('Mystery', 168),
 ('Animation', 139),
 ('Music', 64),
 ('History', 62),
 ('War', 58),
 ('Western', 19),
 ('Documentary', 8),
 ('Foreign', 2)]

ВАРИАНТ 2

In [95]:
genres_dict = {}
for item in data.genres:
    for genre in item:
        if genre in genres_dict.keys():
            genres_dict[genre]+=1
        else:
            genres_dict.setdefault(genre,1)
sorted(genres_dict.items(), key=lambda x:x[1], reverse=True)

[('Drama', 782),
 ('Comedy', 683),
 ('Thriller', 596),
 ('Action', 582),
 ('Adventure', 415),
 ('Crime', 315),
 ('Romance', 308),
 ('Family', 260),
 ('Science Fiction', 248),
 ('Fantasy', 222),
 ('Horror', 176),
 ('Mystery', 168),
 ('Animation', 139),
 ('Music', 64),
 ('History', 62),
 ('War', 58),
 ('Western', 19),
 ('Documentary', 8),
 ('Foreign', 2)]

In [99]:
answers['11'] = 'Drama'

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [101]:
Counter(data[data.profit>0].genres.explode()).most_common()

[('Drama', 560),
 ('Comedy', 551),
 ('Thriller', 446),
 ('Action', 444),
 ('Adventure', 337),
 ('Romance', 242),
 ('Crime', 231),
 ('Family', 226),
 ('Science Fiction', 195),
 ('Fantasy', 188),
 ('Horror', 150),
 ('Animation', 120),
 ('Mystery', 119),
 ('Music', 47),
 ('History', 46),
 ('War', 41),
 ('Western', 12),
 ('Documentary', 7)]

In [102]:
answers['12'] = 'Drama'

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [127]:
data.groupby(['director'])['revenue'].sum().sort_values(ascending=False)

director
Peter Jackson        6490593685
Christopher Nolan    4167548502
David Yates          4154295625
Michael Bay          3886938960
J.J. Abrams          3579169916
                        ...    
David MichÃ´d           2295423
Steven Shainberg        2281089
Paul Schrader           2062066
Keanu Reeves            2054941
Simon Hunter            2033165
Name: revenue, Length: 957, dtype: int64

In [128]:
answers['13'] = 'Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [198]:
data[data['genres'].apply(lambda x:"Action" in x)].explode('director').director.value_counts()

Robert Rodriguez       9
Paul W.S. Anderson     7
Michael Bay            7
Ridley Scott           6
Antoine Fuqua          6
                      ..
Keanu Reeves           1
Douglas Aarniokoski    1
Craig Mazin            1
Patrick Lussier        1
Chris Williams         1
Name: director, Length: 364, dtype: int64

In [199]:
answers['14'] = 'Robert Rodriguez'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [194]:
data[data.release_year==2012].explode('cast').groupby(['cast'])['revenue'].sum().sort_values(ascending=False)

cast
Chris Hemsworth      2027450773
Denis Leary          1629460639
Anne Hathaway        1522851057
Robert Downey Jr.    1519557910
Mark Ruffalo         1519557910
                        ...    
Michael Nyqvist         3428048
Danny Huston            2106557
Josh Lucas              2106557
Sami Gayle              2106557
Nicolas Cage            2106557
Name: revenue, Length: 466, dtype: int64

In [200]:
answers['15'] = 'Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [201]:
data[data.budget>data.budget.mean()].explode('cast').cast.value_counts()

Matt Damon              18
Adam Sandler            17
Angelina Jolie          16
Samuel L. Jackson       15
Tom Cruise              15
                        ..
Aryana Engineer          1
Dave Bautista            1
Jerry Seinfeld           1
Alexander SkarsgÃ¥rd     1
America Ferrera          1
Name: cast, Length: 1505, dtype: int64

In [ ]:
answers['16'] = 'Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [204]:
data[data['cast'].apply(lambda x:"Nicolas Cage" in x)].explode('genres').genres.value_counts()

Action             17
Thriller           15
Drama              12
Crime              10
Fantasy             8
Adventure           7
Comedy              6
Science Fiction     4
Animation           3
Mystery             3
Family              3
History             2
Horror              1
War                 1
Romance             1
Name: genres, dtype: int64

In [205]:
answers['17'] = 'Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [206]:
data[data['production_companies'].apply(lambda x:"Paramount Pictures" in x)].query('profit==profit.min()')

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit
925,tt0267626,100000000,35168966,K-19: The Widowmaker,"[Harrison Ford, Liam Neeson, Peter Sarsgaard, ...",[Kathryn Bigelow],Fate has found its hero.,When Russia's first nuclear submarine malfunct...,138,"[Thriller, Drama, History]","[Paramount Pictures, Intermedia Films, Nationa...",2002-07-19,6.0,2002,-64831034


In [ ]:
answers['18'] = 'K-19: The Widowmaker (tt0267626)'

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [207]:
data.groupby(['release_year'])['revenue'].sum().sort_values(ascending=False)

release_year
2015    25449202382
2014    23405862953
2013    23213799791
2012    23079001687
2011    22676791872
2010    21071204961
2009    20261791024
2008    18252781990
2007    18162406801
2004    15663430720
2005    15309425558
2006    14775042320
2003    14346123312
2002    14136361487
2001    13017764865
2000    10664099805
Name: revenue, dtype: int64

In [26]:
answers['19'] = '2015'

# 20. Какой самый прибыльный год для студии Warner Bros?

In [25]:
df = data.explode('production_companies')
df[df['production_companies'].str.contains("Warner Bros")].groupby('release_year')['profit'].sum().sort_values(ascending=False)

release_year
2014    2292949646
2007    2201675217
2008    2134595031
2010    1974712985
2011    1871393682
2003    1844008221
2009    1822454136
2013    1636453400
2004    1631933725
2005    1551980298
2001    1282142579
2012    1258020056
2002    1022709901
2015     870368348
2006     625351872
2000     452631386
Name: profit, dtype: int64

In [27]:
answers['20'] = '2014'

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [30]:
data.groupby(data['release_date'].dt.month)['imdb_id'].count().sort_values(ascending=False)

release_date
9     227
12    190
10    186
8     161
3     156
4     149
6     147
11    146
7     142
5     140
2     135
1     110
Name: imdb_id, dtype: int64

In [ ]:
answers['21'] = 'Сентябрь'

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [33]:
data.query('release_date.dt.month in [6,7,8]').imdb_id.count()

450

In [34]:
answers['22'] = '450'

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [36]:
df = data.explode('director')
df.query('release_date.dt.month in [1,11,12]').groupby('director')['imdb_id'].count().sort_values(ascending=False)

director
Peter Jackson        8
Clint Eastwood       7
Steven Soderbergh    6
Nancy Meyers         4
Adam Shankman        4
                    ..
James Gunn           1
James Duffy          1
James Cameron        1
James Bobin          1
Xavier Gens          1
Name: imdb_id, Length: 371, dtype: int64

In [ ]:
answers['23'] = 'Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [65]:
df = data.explode('production_companies')
df.groupby('production_companies').original_title.apply(lambda x:x.str.len().max()).sort_values(ascending=False)

production_companies
Twentieth Century Fox Film Corporation    83
Four By Two Productions                   83
Walt Disney                               62
Walden Media                              62
21 Laps Entertainment                     59
                                          ..
Everest Entertainment                      3
Berlanti Productions                       3
Global Entertainment Group                 2
XM2 Productions                            2
Ixtlan Productions                         2
Name: original_title, Length: 1771, dtype: int64

In [60]:
data.explode('production_companies').query('production_companies.str.contains("Four By Two")')

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit
1448,tt0443453,18000000,261572744,Borat: Cultural Learnings of America for Make ...,"[Sacha Baron Cohen, Ken Davitian, Luenell, Pam...",[Larry Charles],"Come to Kazakhstan, it's nice!",Kazakh journalist Borat Sagdiyev travels to Am...,82,[Comedy],Four By Two Productions,2006-11-02,6.4,2006,243572744


In [57]:
data[data.imdb_id=='tt0443453'].production_companies.values[0]

['Twentieth Century Fox Film Corporation', 'Four By Two Productions']

In [59]:
data[data.imdb_id=='tt0443453'].original_title.values[0]

'Borat: Cultural Learnings of America for Make Benefit Glorious Nation of Kazakhstan'

In [64]:
len(data[data.imdb_id=='tt0443453'].original_title.values[0])

83

In [ ]:
answers['24'] = 'Four By Two Productions'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [71]:
s=data.vote_average.value_counts(bins=100)
s.sort_index()

(3.294, 3.348]    2
(3.348, 3.396]    0
(3.396, 3.444]    2
(3.444, 3.492]    0
(3.492, 3.54]     2
                 ..
(7.86, 7.908]     9
(7.908, 7.956]    0
(7.956, 8.004]    4
(8.004, 8.052]    0
(8.052, 8.1]      1
Name: vote_average, Length: 100, dtype: int64

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


ВАРИАНТ 2

# Submission

In [0]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': '...', '2': '...'}

In [0]:
# и убедиться что ни чего не пропустил)
len(answers)